In [ ]:
from urbansim.models.dcm import MNLDiscreteChoiceModel

import os
import numpy as np
import pandas as pd

import orca
from urbansim.utils import misc

import dataset
import variables
import models
import utils

orca.run(["build_networks"])
orca.run(["neighborhood_vars"])

### Estimate HLCM from config

In [2]:
hlcm1_model = MNLDiscreteChoiceModel.from_yaml(str_or_buffer=misc.config('hlcm/hlcm1.yaml'))

hh = orca.get_table('households').to_frame(hlcm1_model.columns_used())
b = orca.get_table('buildings').to_frame(hlcm1_model.columns_used())

hh = hh[hh.income_quartile == 1]
b = b.fillna(0)
hh = hh.fillna(0)

print hlcm1_model.fit(hh, b, hh[hlcm1_model.choice_column])

hlcm1_model.fit_parameters

Disaggregating drv_60min_jobs to buildings from nodes_drv
Disaggregating ave_nonres_sqft_price to buildings from nodes_walk
Disaggregating retail_jobs to buildings from nodes_walk
Disaggregating ln_popden to buildings from nodes_walk
Disaggregating a_ln_emp_50min_transit to buildings from zones
Disaggregating percent_race1 to buildings from nodes_walk
Disaggregating percent_hh_with_children to buildings from nodes_walk


/home/janowicz/udst/urbansim/urbansim/urbanchoice/interaction.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  alts_sample['join_index'] = np.repeat(choosers.index.values, SAMPLE_SIZE)


{'convergence': -11103.158068510478, 'null': -13815.51055796495, 'ratio': 0.19632661985776123}


,Coefficient,Std. Error,T-Score
residential_units,0.020587,0.000719,28.640595
b_is_pre_1945,0.508882,0.039600,12.850403
has_children:nodes_walk_percent_hh_with_children,3.000000,0.193764,15.482763
has_workers:zones_a_ln_emp_50min_transit,0.026849,0.003979,6.747746
drv_nearest_healthcenter,-0.082261,0.005307,-15.499156
b_ln_land_area,1.060727,0.019718,53.793863
nodes_drv_drv_60min_jobs,0.015205,0.011741,1.295049
is_race1:nodes_walk_percent_race1,3.000000,0.071664,41.861785
persons:nodes_walk_ln_popden,0.153513,0.006905,22.231100
nodes_walk_ave_nonres_sqft_price,0.000807,0.000319,2.526350


## Hedonic Estimation

In [ ]:
orca.run(["rsh_estimate"])

In [ ]:
orca.run(["nrh_estimate"])

## LCM Estimation

In [ ]:
orca.run(["hlcm_estimate"])

In [ ]:
orca.run(["elcm_estimate"])